In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *


## select()

In [3]:
# DataFrame.select(*cols)[source]
# Parameters
# cols : str, Column, or list
# column names (string) or expressions (Column). 
#If one of the column names is ‘*’, that column is expanded to include all columns in the current DataFrame.

class_df = spark.read.csv('/dataframe/a_class_info.csv', header=True)
class_df

DataFrame[class_cd: string, school: string, class_std_cnt: string, loc: string, school_type: string, teaching_type: string]

In [34]:
class_df.columns
class_df1=class_df.select('class_cd', 'school', class_df.class_std_cnt.cast('int'), 'loc', 'school_type', 'teaching_type')

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

In [8]:
# str 
# DataFrame.select(*cols: ColumnOrName)
# col을 무한으로 넣을 수 있고 dataframe 객체를 반환
df_spark = class_df.select('school','loc')
df_spark.show(3)

# *
df_spark = class_df.select('*')
df_spark.show(3)

# list
df_spark = class_df.select(['school','loc'])
df_spark.show(3)

+------+--------+
|school|     loc|
+------+--------+
| ANKYI|   Urban|
| ANKYI|   Urban|
| CCAAW|Suburban|
+------+--------+
only showing top 3 rows



+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



+------+--------+
|school|     loc|
+------+--------+
| ANKYI|   Urban|
| ANKYI|   Urban|
| CCAAW|Suburban|
+------+--------+
only showing top 3 rows



#### Column 

In [4]:
# Colunm : 내부에 DataFrame 연산을 위한 Expression을 가지고 있는 객체
col('school')


# 데이터프레임에서 원하는 컬럼을 반환
class_df.school

# 정규표현식 : Column<'unresolvedregex()'> : 수행되지않은 정규표현식
class_df.colRegex("`^school.*`")

# list함수로 컬럼객체 반환
lit('school')

#

Column<'school'>

Column<'school'>

Column<'unresolvedregex()'>

Column<'school'>

In [26]:
class_df = class_df.select(col('school'))
class_df.show(3)

class_df = class_df.select(class_df.colRegex("`^school.*`"))
class_df.show(3)

class_df = class_df.select(lit('school'))
class_df.show(3)


# 다른 데이터프레임 객체에서만 df이름.column이름 으로 해당 컬럼을 불러올 수 있음 같은 데이터 프레임에만 사용 가능
class_df = class_df.select(class_df.school)
class_df.show(3)



+------+
|school|
+------+
|school|
|school|
|school|
+------+
only showing top 3 rows



+------+
|school|
+------+
|school|
|school|
|school|
+------+
only showing top 3 rows



+------+
|school|
+------+
|school|
|school|
|school|
+------+
only showing top 3 rows



+------+
|school|
+------+
|school|
|school|
|school|
+------+
only showing top 3 rows



#### Column 연산

- 산술연산 :  +  -  *  /  %
- 비교연산 : <  >  <=  >=  ==  !=
- 논리연산 : &  |




![op-overloading](./img/op_overloading.png)

In [9]:
# 10을 더한다는 말은 class_std_cnt라는 컬럼객체의 각셀에 10을 더한다는 
# 컬럼에다 값을 저장한다는게 아니고
# 컬럼을 호출할 논리계획을 세우는 것
# 아래연산의 실행결과는 column객체
class_df.class_std_cnt+100
class_df.class_std_cnt  - 100
class_df.class_std_cnt  / 100
class_df.class_std_cnt  % 100
class_df.class_std_cnt  < 100
class_df.class_std_cnt  > 100
class_df.class_std_cnt  <= 100
(class_df.class_std_cnt  >= 100) &(class_df.class_std_cnt  >= 1000)

# DataFrame의 select 메서드에서 새로운 컬럼을 전달인자로 보낼 경우 해당 컬림이 추가된 새로운 DataFrame이 생성된다.
class_df.select(class_df.school, class_df.class_cd, (class_df.class_std_cnt  >= 10) &(class_df.class_std_cnt  >= 20)).show(3)
#  컬럼이름이 가독성 떨어짐 : columnname : (class_df.class_std_cnt  >= 10) &(class_df.class_std_cnt  >= 20) 
# alias() 함수로 컬럼이름 바꿔줌 또한 해당 함수는 column 객체를 반환 이 객체를 변수에 선언하여 사용 가능
# Column.alias(*alias: str, **kwargs: Any) → pyspark.sql.column.Column
col_is_big = ((class_df.class_std_cnt  >= 10) &(class_df.class_std_cnt  >= 20)).alias('is_big')
# 결과

class_df.select(class_df.school, class_df.class_cd, col_is_big).show(3)
class_df.select(class_df.school, class_df.class_cd, (class_df.class_std_cnt <10).alias('10이하')) \
            .where(class_df.school.like('%N%')).show(3)

Column<'(class_std_cnt + 100)'>

Column<'(class_std_cnt - 100)'>

Column<'(class_std_cnt / 100)'>

Column<'(class_std_cnt % 100)'>

Column<'(class_std_cnt < 100)'>

Column<'(class_std_cnt > 100)'>

Column<'(class_std_cnt <= 100)'>

Column<'((class_std_cnt >= 100) AND (class_std_cnt >= 1000))'>

+------+--------+-------------------------------------------------+
|school|class_cd|((class_std_cnt >= 10) AND (class_std_cnt >= 20))|
+------+--------+-------------------------------------------------+
| ANKYI|     6OL|                                             true|
| ANKYI|     ZNS|                                             true|
| CCAAW|     2B1|                                            false|
+------+--------+-------------------------------------------------+
only showing top 3 rows

+------+--------+------+
|school|class_cd|is_big|
+------+--------+------+
| ANKYI|     6OL|  true|
| ANKYI|     ZNS|  true|
| CCAAW|     2B1| false|
+------+--------+------+
only showing top 3 rows



+------+--------+------+
|school|class_cd|10이하|
+------+--------+------+
| ANKYI|     6OL| false|
| ANKYI|     ZNS| false|
| DNQDD|     2AP| false|
+------+--------+------+
only showing top 3 rows



#### Column 메서드

In [47]:
school = class_df.school # 컬럼객체 셀렉트 문에 활용가능
class_cd = class_df.class_cd
std_cnt = class_df.class_std_cnt

std_cnt .asc() # 컬럼객체
# class_df.select(school, class_cd, std_cnt.asc()).show(3) # 이 구문은 오류 

# 아래 코드로 사용
# DataFrame.sort(*cols: Union[str, pyspark.sql.column.Column, List[Union[str, pyspark.sql.column.Column]]], **kwargs: Any) 
# → pyspark.sql.dataframe.DataFrame

# asc()는 sort된 DataFrame객체를 기반으로 작동하기 때문에 select()에 바로 넣으면 오류가 난다.
class_df.select(school, class_cd,std_cnt).sort(std_cnt.asc()).show(100) 
# Column<'class_std_cnt ASC NULLS FIRST'> 

# 내림차순정렬
# Column.desc() → pyspark.sql.column.Column
class_df.select(school, class_cd,std_cnt).sort(std_cnt.desc()).show(100) 

Column<'class_std_cnt ASC NULLS FIRST'>

+------+--------+-------------+
|school|class_cd|class_std_cnt|
+------+--------+-------------+
|  null|     4SZ|         null|
|  null|     6PP|         null|
|  null|     5SD|         null|
|  null|     MDE|           10|
| FBUMG|     197|           14|
| FBUMG|     JGD|           14|
| VHDHF|     KR1|           15|
| CCAAW|     IQN|           15|
| UUUQX|     SSP|           15|
| CCAAW|     UHU|           16|
| UUUQX|     H7S|           16|
| UUUQX|     6C1|           16|
| VHDHF|     J6X|           16|
| IDGFP|     P2A|           17|
| IDGFP|     XZM|           17|
| LAYPA|     RA5|           17|
| CIMBB|     HUJ|           17|
| UUUQX|     AE1|           17|
| IDGFP|     G2L|           17|
| OJOBU|     5SZ|           17|
| CIMBB|     PC6|           17|
| UKPGS|     KXB|           18|
| VKWQH|     341|           18|
| CCAAW|     2B1|           18|
| FBUMG|     5LQ|           18|
|  null|     MDS|           18|
| GOKXL|     VA6|           19|
| UKPGS|     IPU|           19|
| CIMBB|

+------+--------+-------------+
|school|class_cd|class_std_cnt|
+------+--------+-------------+
| GOOBU|     18K|           31|
| VVTVA|     A93|           30|
| VVTVA|     YTB|           30|
| ZOWMK|     Q0E|           30|
| ZOWMK|     QA2|           30|
| ZOWMK|     ZBH|           30|
| VVTVA|     7BL|           29|
| CUQAM|     1Q1|           28|
| QOQTS|     0N7|           28|
| CUQAM|     OMI|           28|
| DNQDD|     ROP|           28|
| GOOBU|     HKF|           28|
| QOQTS|     SUR|           28|
| GJJHK|     NOR|           27|
| GJJHK|     ZDT|           27|
| KFZMY|     1VD|           27|
| ZOWMK|     CII|           27|
| CUQAM|     BFY|           27|
| DNQDD|     XXJ|           27|
| DNQDD|     2AP|           27|
| GOOBU|     W8A|           26|
| UKPGS|     9AW|           25|
| VVTVA|     TB5|           25|
| GOOBU|     U6J|           25|
| KFZMY|     21Q|           25|
| QOQTS|     X2O|           25|
| GOOBU|     PBA|           24|
| CUQAM|     X6Z|           24|
| GOOBU|

## select 연습

DataFrame은 class_df 를 사용합니다

1. 모든 row의  class_cd, school, loc, school_type, teaching_type을 출력하시오.  
   school_type의 컬럼명은 '공립/사립여부' 로 표시합니다.
   
   
2. class_cd, school, scale 을 출력하시오.   
   scale 컬럼은 반의 인원수가 15명 미만이면 small, 15명 이상 25명 미만이면 middle, 25명 이상이면 big 값을 가집니다.
   
   
3. class_cd, school, class_std_cnt를 출력하시오  
   class_std_cnt 컬럼의 값들은 string타입으로 형변환합니다.

In [27]:
# class_df.class_cd
# class_df.school
# class_df.loc
# class_df.school_type
# class_df.teaching_type

# 모든 row의 class_cd, school, loc, school_type, teaching_type을 출력하시오.
#school_type의 컬럼명은 '공립/사립여부' 로 표시합니다.
# class_df.select(class_df.class_cd,class_df.school,class_df.loc,class_df.school_type.alias('공립/사립여부'),class_df.teaching_type).show(3)


# class_cd, school, scale 을 출력하시오.
# scale 컬럼은 반의 인원수가 15명 미만이면 small, 15명 이상 25명 미만이면 middle, 25명 이상이면 big 값을 가집니다.
# class_df.select(class_df.class_cd, class_df.school, class_df.scale)
class_df1 = class_df.withColumn('scale', when(class_df.class_std_cnt < 25 , 'middle')
                   .when(class_df.class_std_cnt < 15, 'small')
                   .otherwise('big'))
class_df1.select(class_df1.class_cd, class_df1.school, class_df1.scale).show(3)

# lass_cd, school, class_std_cnt를 출력하시오
# class_std_cnt 컬럼의 값들은 string타입으로 형변환합니다.
# class_df.class_std_cnt.cast('string')
class_df.select(class_df.class_cd, class_df.class_std_cnt.cast('int')).show(3)
class_df.select(class_df.class_cd, class_df.class_std_cnt.cast('int'))






+--------+------+------+
|class_cd|school| scale|
+--------+------+------+
|     6OL| ANKYI|middle|
|     ZNS| ANKYI|middle|
|     2B1| CCAAW|middle|
+--------+------+------+
only showing top 3 rows



+--------+-------------+
|class_cd|class_std_cnt|
+--------+-------------+
|     6OL|           20|
|     ZNS|           21|
|     2B1|           18|
+--------+-------------+
only showing top 3 rows



In [4]:
from pyspark.sql import functions as F
cdf = class_df

# 모든 row의 class_cd, school, loc, school_type, teaching_type을 출력하시오.
# school_type의 컬럼명은 '공립/사립여부' 로 표시합니다.
# hint : alias()
class_df.select('class_cd', 'school', 'loc', class_df.school_type.alias('공립/사립여부'), 'teaching_type').show(3)

# class_cd, school, scale 을 출력하시오.
# scale 컬럼은 반의 인원수가 15명 미만이면 small, 15명 이상 25명 미만이면 middle, 25명 이상이면 big 값을 가집니다.
# hint : when()
class_df.printSchema()
class_df.select('class_cd', 'school', (when(class_df.class_std_cnt <15 , 'small')
                                                   .when(class_df.class_std_cnt.between(15,24),'middle')
#                                                    .when((class_df.class_std_cnt >= 15) & (class_df.class_std_cnt< 25), 'middle')
                                                   .otherwise('big')).alias('scale')).show(20)


# class_cd, school, class_std_cnt를 출력하시오
# class_std_cnt 컬럼의 값들은 string타입으로 형변환합니다.
# hint : cast()
class_df.select('class_cd', 'school', class_df.class_std_cnt.cast("string")).show(3)
class_df.select('class_cd', 'school', class_df.class_std_cnt.cast("string")).printSchema()


+--------+------+--------+-------------+-------------+
|class_cd|school|     loc|공립/사립여부|teaching_type|
+--------+------+--------+-------------+-------------+
|     6OL| ANKYI|   Urban|   Non-public|     Standard|
|     ZNS| ANKYI|   Urban|   Non-public|     Standard|
|     2B1| CCAAW|Suburban|   Non-public| Experimental|
+--------+------+--------+-------------+-------------+
only showing top 3 rows

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



+--------+------+------+
|class_cd|school| scale|
+--------+------+------+
|     6OL| ANKYI|middle|
|     ZNS| ANKYI|middle|
|     2B1| CCAAW|middle|
|     EPS| CCAAW|middle|
|     IQN| CCAAW|middle|
|     PGK| CCAAW|middle|
|     UHU| CCAAW|middle|
|     UWK| CCAAW|middle|
|     A33| CIMBB|middle|
|     EID| CIMBB|middle|
|     HUJ| CIMBB|middle|
|     PC6| CIMBB|middle|
|     1Q1| CUQAM|   big|
|     BFY| CUQAM|   big|
|     OMI| CUQAM|   big|
|     X6Z| CUQAM|middle|
|     2AP| DNQDD|   big|
|     PW5| DNQDD|middle|
|     ROP| DNQDD|   big|
|     ST7| DNQDD|middle|
+--------+------+------+
only showing top 20 rows



+--------+------+-------------+
|class_cd|school|class_std_cnt|
+--------+------+-------------+
|     6OL| ANKYI|           20|
|     ZNS| ANKYI|           21|
|     2B1| CCAAW|           18|
+--------+------+-------------+
only showing top 3 rows

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)



### sql

In [40]:
# sql을 사용해 같은 결과를 받아보기

# dataFrame을 테이블로 등록
class_df1.createOrReplaceTempView('class')
spark.sql('''
    select * from class
''').printSchema()
# 모든 row의 class_cd, school, loc, school_type, teaching_type을 출력하시오.
# school_type의 컬럼명은 '공립/사립여부' 로 표시합니다.
spark.sql('''
    select class_cd, school, loc, school_type as `공립/사립여부`, teaching_type from class
''').show(3)


# class_cd, school, scale 을 출력하시오.
# scale 컬럼은 반의 인원수가 15명 미만이면 small, 15명 이상 25명 미만이면 middle, 25명 이상이면 big 값을 가집니다.
spark.sql('''
    SELECT class_cd, school, 
    CASE WHEN class_std_cnt < 15 THEN 'small' 
    WHEN class_std_cnt between 15 and 24 THEN 'middle' 
    else 'big'
    END scale 
    FROM class
''').show(200)


# class_cd, school, class_std_cnt를 출력하시오
# class_std_cnt 컬럼의 값들은 string타입으로 형변환합니다.
# spark built-in-fnc 의 cast()
spark.sql('''
    select class_cd, school, cast(class_std_cnt as int) from class
''').printSchema()


root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

+--------+------+--------+-------------+-------------+
|class_cd|school|     loc|공립/사립여부|teaching_type|
+--------+------+--------+-------------+-------------+
|     6OL| ANKYI|   Urban|   Non-public|     Standard|
|     ZNS| ANKYI|   Urban|   Non-public|     Standard|
|     2B1| CCAAW|Suburban|   Non-public| Experimental|
+--------+------+--------+-------------+-------------+
only showing top 3 rows



+--------+------+------+
|class_cd|school| scale|
+--------+------+------+
|     6OL| ANKYI|middle|
|     ZNS| ANKYI|middle|
|     2B1| CCAAW|middle|
|     EPS| CCAAW|middle|
|     IQN| CCAAW|middle|
|     PGK| CCAAW|middle|
|     UHU| CCAAW|middle|
|     UWK| CCAAW|middle|
|     A33| CIMBB|middle|
|     EID| CIMBB|middle|
|     HUJ| CIMBB|middle|
|     PC6| CIMBB|middle|
|     1Q1| CUQAM|   big|
|     BFY| CUQAM|   big|
|     OMI| CUQAM|   big|
|     X6Z| CUQAM|middle|
|     2AP| DNQDD|   big|
|     PW5| DNQDD|middle|
|     ROP| DNQDD|   big|
|     ST7| DNQDD|middle|
|     XXJ| DNQDD|   big|
|     197| FBUMG| small|
|     5LQ| FBUMG|middle|
|     JGD| FBUMG| small|
|     HCB| GJJHK|middle|
|     NOR| GJJHK|   big|
|     X78| GJJHK|middle|
|     YUC| GJJHK|middle|
|     ZDT| GJJHK|   big|
|     ENO| GOKXL|middle|
|     TSA| GOKXL|middle|
|     VA6| GOKXL|middle|
|     18K| GOOBU|   big|
|     CXC| GOOBU|middle|
|     HKF| GOOBU|   big|
|     PBA| GOOBU|middle|
|     U6J| GOOBU|   big|


## where(), filter()

1. 공립이면서 교육방식이 전문인 데이터를 출력하시오

2. 사립이면서 교육방식이 표준인 데이터를 출력하시오

3. 학교 이름이 V로 시작하는 데이터를 출력하시오

4. 학교 이름이 M로 끝나는 데이터를 출력하시오

5. 학교 이름에 NKY가 포함된 데이터를 출력하시오

6. 반의 학생 수가 15명 이상 24명 이하인 데이터를 출력하시오

7. 학교 이름이 입력되지 않은 데이터들을 출력하시오


In [7]:
class_df.printSchema()

print('공립이면서 교육방식이 전문인 데이터를 출력하시오.')
class_df.select('*') \
            .where((class_df.school_type =="Public") &(class_df.teaching_type =='Experimental')) \
            .show(3)
print('사립이면서 교육방식이 표준 데이터를 출력하시오.')
class_df.select('*') \
            .where((class_df.school_type=='Non-public')&(class_df.teaching_type =='Standard')) \
            .show(3)
print('학교 이름이 V로 시작하는 데이터를 출력하시오.')
# class_df.select('*').where(col('school').like("V%")).show(3)
class_df.select('*') \
            .where(class_df.school.startswith('V')) \
            .show(3)
print('학교 이름이 M로 끝하는 데이터를 출력하시오.')
# class_df.select('*').where((class_df.school).like("%M")).show(3)
class_df.select('*') \
            .where(class_df.school.endswith('M')) \
            .show(3)
print('학교 이름에 NKY가 들어가는 데이터를 출력하시오.')
class_df.select('*').where((class_df.school).like("%NKY%")).show(3)
class_df.slelect('*') \
            .where(class_df.school.contains('NKY')) \
            .show(3)
print('반의 학생 수가 15명 이상 24명 이하인 데이터를 출력하시오.')
# class_df.select('*').where((class_df.class_std_cnt>= 15) & (class_df.class_std_cnt < 24)).show(3)
class_df.select('*').where(class_df.class_std_cnt.between(15,24)).show(3)
print('학교 이름이 입력되지 않은 데이터들을 출력하시오')

class_df.select('*').where(isnull(class_df.school)).show(3)
class_df.select('*') \
            .where(class_df.school.isNull()) \
            .show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

공립이면서 교육방식이 전문인 데이터를 출력하시오.


+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     X6Z| CUQAM|           24|   Urban|     Public| Experimental|
|     PW5| DNQDD|           20|Suburban|     Public| Experimental|
|     ROP| DNQDD|           28|Suburban|     Public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows

사립이면서 교육방식이 표준 데이터를 출력하시오.


+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows

학교 이름이 V로 시작하는 데이터를 출력하시오.


+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     CD8| VHDHF|           20|Rural| Non-public| Experimental|
|     J6X| VHDHF|           16|Rural| Non-public|     Standard|
|     KR1| VHDHF|           15|Rural| Non-public| Experimental|
+--------+------+-------------+-----+-----------+-------------+
only showing top 3 rows

학교 이름이 M로 끝하는 데이터를 출력하시오.


+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     1Q1| CUQAM|           28|Urban|     Public|     Standard|
|     BFY| CUQAM|           27|Urban|     Public|     Standard|
|     OMI| CUQAM|           28|Urban|     Public|     Standard|
+--------+------+-------------+-----+-----------+-------------+
only showing top 3 rows

학교 이름에 NKY가 들어가는 데이터를 출력하시오.
+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     6OL| ANKYI|           20|Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|Urban| Non-public|     Standard|
+--------+------+-------------+-----+-----------+-------------+



AttributeError: 'DataFrame' object has no attribute 'slelect'

#### sql

In [37]:
print('공립이면서 교육방식이 전문인 데이터를 출력하시오')
spark.sql(
    "SELECT * FROM class WHERE school_type = {bound1} AND teaching_type = {bound2}", bound1='Public', bound2='Experimental'
).show()
print('사립이면서 교육방식이 표준인 데이터를 출력하시오')
# spark.sql(’”
#           select * from class
#           where school_type = `Non-public` and teaching_type = `Standard'
#           ”’)
print('학교 이름이 V로 시작하는 데이터를 출력하시오')

print('학교 이름이 M로 끝나는 데이터를 출력하시오')

print('학교 이름에 NKY가 포함된 데이터를 출력하시오')

print('반의 학생 수가 15명 이상 24명 이하인 데이터를 출력하시오')

          
print('학교 이름이 입력되지 않은 데이터들을 출력하시오')


공립이면서 교육방식이 전문인 데이터를 출력하시오


TypeError: sql() got an unexpected keyword argument 'bound1'

In [38]:
print(1)

1
